# **PRECIO PROMEDIO POR BAÑO**

In [10]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre

    #if nombre.startswith("df_") and "pachuca" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [11]:
df_alfa_junio_2024_pachuca.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [12]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Preciol_23':'precio','Baño_total': 'baño_total','Precio':'precio','Banos_Total':'baño_total'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [13]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [14]:
import numpy as np
for df in dataframes_list:    
    condition = (df['baño_total'] % 1 == 0.5) # Valores que contienen .5    
    # Convertir al número entero anterior
    df.loc[condition, 'baño_total'] = np.floor(df['baño_total'])    
    print(df['baño_total'].unique())

[1 7 2 3 6 4 5]
[ 3.  4.  2.  1.  5.  7. 21.  6.  8.  9.  0.]
[6. 5. 8. 2. 1. 4. 7. 3. 9. 0.]
[ 2.  1.  3.  4.  0.  5. 21.  6.  7.  9.  8.]
[1 3 2 4 0 5 6 7 9 8]
[6 4 8 1 3 2 7 5]
[2. 1. 4. 3. 5. 7. 9. 6. 8.]
[1 2 3 0 4 5 6 7]
[1 2 4 3 5 6]
[1 2 3 4 5 6 7 8 9]
[ 1  2  3  4 13  5  8  6  7]
[ 1  2  3  0  4  5  6  9  7  8 22]
[ 1  2  3  0  4  5  6  9  7  8 22]
[ 3  2  1  4  5  6  7 10  8  9]


C:\Users\yoe11\AppData\Local\Temp\ipykernel_6164\4078948052.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
# Concatenar todos los DataFrames individuales en uno solo
df_concatenado = pd.concat(dataframes_list)

# Ajustar los valores de 'baño_total' para que no superen 6
df_concatenado['baño_total'] = df_concatenado['baño_total'].clip(lower=1,upper=6)
# Calcular el promedio de precio por el número único de baños
df_promedio_baños = df_concatenado.groupby('baño_total')['precio'].mean().reset_index()
df_promedio_baños

,baño_total,precio
0,1.0,2.226130e+06
1,2.0,2.054665e+06
2,3.0,1.994983e+06
3,4.0,2.484049e+06
4,5.0,2.407700e+06
5,6.0,3.009254e+06


In [16]:
import pandas as pd
import plotly.graph_objects as go
import locale

# Establecer configuración regional para el formato de precios
locale.setlocale(locale.LC_ALL, '')

# Definir función para formatear precios
def format_price(price):
    return locale.currency(price, grouping=True)

# Crear una lista de etiquetas para el eje x
etiquetas = [str(baños) if baños != 6 else '6+' for baños in df_promedio_baños['baño_total']]

# Obtener los precios y el número de baños
precios = df_promedio_baños['precio']

# Crear el texto con los precios formateados
text_precios = [format_price(precio) for precio in precios]

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D','#2962ff', '#9500ff', '#ff0059','#2962ff', ]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=etiquetas,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))
fig.update_layout(
    #title='Precio promedio por número de baños',
    xaxis=dict(title='Baños'),
    yaxis=dict(title='$', gridcolor='#dddcda', ),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_banos',  carpeta='assets/graficas')
fig.show()

### PRECIO PROMEDIO POR PROPIEDAD (1-5 BAÑOS)
METER .5 PARA EL ANTERIOR 

In [17]:
df_promedio_baños

,baño_total,precio
0,1.0,2.226130e+06
1,2.0,2.054665e+06
2,3.0,1.994983e+06
3,4.0,2.484049e+06
4,5.0,2.407700e+06
5,6.0,3.009254e+06


In [18]:
# Filtrar solo los registros con baño_total igual a 1, 2, 3 o 4
df_promedio_baños_filtrado = df_promedio_baños[df_promedio_baños['baño_total'].isin([1, 2, 3, 4])]
df_promedio_baños_filtrado

,baño_total,precio
0,1.0,2.226130e+06
1,2.0,2.054665e+06
2,3.0,1.994983e+06
3,4.0,2.484049e+06


In [19]:
import numpy as np

# Redondear los valores de baños al entero más cercano
baños_redondeados = np.round(df_promedio_baños_filtrado['baño_total']).astype(int)

# Crear la gráfica de barras con los valores redondeados
fig = go.Figure()
fig.add_trace(go.Bar(
    x=baños_redondeados,
    y=precios,
    text=text_precios,
    marker_color=colores,
    textposition='auto' 
))

# Actualizar el diseño del gráfico
fig.update_layout(
    #title='Precio Promedio de la Propiedad por Número de Baños',
    xaxis=dict(title='Baños', tickvals=np.arange(baños_redondeados.min(), baños_redondeados.max() + 1)),
    yaxis=dict(title='$',gridcolor='#dddcda',),
    bargap=0.1,
    bargroupgap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_banos_14',  carpeta='assets/graficas')
fig.show()